# **Lab 11: PySpark Dataframes and Spark SQL**

Remember in Lab 10, we did include the following description of SparkSession. 

- **SparkSession** serves as entry point for Spark 2.0 and later. It combines the functionality of SparkContext, SQLContext, HiveContext and StreamingContext. In another words, you can work on DataFrames and Datasets, perform structured query (SQL), Machine Learning algorithms and streaming. In this notebook, we will look into the **working on DataFrames and SQL** that is supported through Spark Session. 

# [Pyspark Dataframes](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html)

A distributed collection of data grouped into named columns.A DataFrame is equivalent to a relational table in Spark SQL, and can be created using various functions in SparkSession. Once created, it can be manipulated using the various domain-specific-language (DSL) functions.  

Credit goes to [website](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/) for examples of Pyspark dataframe operations. 

### So, is there any differences between Lab10's Pyspark RDD and Pyspark DataFrames?  
From an [article](https://sachee.medium.com/apache-spark-dataframe-vs-rdd-24a04d2eb1b9):

*   Pyspark DataFrames require a schema and you can think of them as “tables” of data. RDDs are less structured and closer to Scala collections or lists.
*   Operations on Pyspark DataFrames are optimizable by Spark whereas operations on RDDs are imperative and run through the transformations and actions in order.





## Spark dataframes are immutable
When a Spark dataframe is created,the content of the dataframe cannot be changed but a new dataframe will be created. Internally in Spark, commands such as filter() will not change the dataframe but it creates a new dataframe from the output of executing the command. 

Like an RDD, a DataFrame is an immutable distributed collection of data. Unlike an RDD, data is organized into named columns, like a table in a relational database. Designed to make large data sets processing even easier, DataFrame allows developers to impose a structure onto a distributed collection of data, allowing higher-level abstraction; it provides a domain specific language API to manipulate your distributed data; and makes Spark accessible to a wider audience, beyond specialized data engineers.

### What about the dataframe that we used in previous labs  such as Lab02 and etc?
In previous labs, we did run dataframe but remember it is Python's dataframe where we imported the pandas library. It is not Pyspark dataframe.  



In [1]:
# Set the PySpark environment variables. 
# The Spark's path depends on where you have saved your Spark. Refer to earlier guide in PDF  
import os
os.environ['SPARK_HOME'] = "/Applications/Spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'notebook'
os.environ['PYSPARK_PYTHON'] = 'python'

In [2]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("MyDFandSQL") \
    .getOrCreate()

# Get the SparkContext from the SparkSession
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/16 23:21:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Let's read the data that we have used in Lab03. 

In [3]:
imdb_df = spark.read.csv("IMDB_top250.csv",header=True)

Let's try to understand the data. First, we need to know the column names and types. 

In [4]:
imdb_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Rated: string (nullable = true)
 |-- Released: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Writer: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Plot: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Awards: string (nullable = true)
 |-- Poster: string (nullable = true)
 |-- Ratings.Source: string (nullable = true)
 |-- Ratings.Value: string (nullable = true)
 |-- Metascore: string (nullable = true)
 |-- imdbRating: string (nullable = true)
 |-- imdbVotes: string (nullable = true)
 |-- imdbID: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- tomatoMeter: string (nullable = true)
 |-- tomatoImage: string (nullable = true)
 |-- tomatoRating: string (nullable = true)
 |-- tom

In [5]:
imdb_df.show(5)  # What is the function used to retrieve the first 5 rows of a Python's data frame?

24/09/16 23:21:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/09/16 23:21:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, Year, Rated, Released, Runtime, Genre, Director, Writer, Actors, Plot, Language, Country, Awards, Poster, Ratings.Source, Ratings.Value, Metascore, imdbRating, imdbVotes, imdbID, Type, tomatoMeter, tomatoImage, tomatoRating, tomatoReviews, tomatoFresh, tomatoRotten, tomatoConsensus, tomatoUserMeter, tomatoUserRating, tomatoUserReviews, tomatoURL, DVD, BoxOffice, Production, Website, Response
 Schema: _c0, Title, Year, Rated, Released, Runtime, Genre, Director, Writer, Actors, Plot, Language, Country, Awards, Poster, Ratings.Source, Ratings.Value, Metascore, imdbRating, imdbVotes, imdbID, Type, tomatoMeter, tomatoImage, tomatoRating, tomatoReviews, tomatoFresh, tomatoRotten, tomatoConsensus, tomatoUserMeter, t

+---+--------------------+----+--------+-----------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+---------+----------+---------+---------+---------+-----------+-----------+------------+-------------+-----------+------------+---------------+---------------+----------------+-----------------+--------------------+--------------------+------------+--------------------+--------------------+--------+
|_c0|               Title|Year|   Rated|   Released|Runtime|               Genre|            Director|              Writer|              Actors|                Plot|            Language|Country|              Awards|              Poster|      Ratings.Source|       Ratings.Value|Metascore|imdbRating|imdbVotes|   imdbID|     Type|tomatoMeter|tomatoImage|tomatoRating|tomatoReviews|tomatoFresh|tomatoRotten|tomatoConsensus|

Let's calculate the total number of rows.

In [6]:
imdb_df.count()

250

In [7]:
imdb_df.show(truncate=False)
#What happens when truncate = True ?

+---+-------------------------------------------------+----+--------+-----------+-------+--------------------------+-------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+----------------+--------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------

24/09/16 23:21:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, Year, Rated, Released, Runtime, Genre, Director, Writer, Actors, Plot, Language, Country, Awards, Poster, Ratings.Source, Ratings.Value, Metascore, imdbRating, imdbVotes, imdbID, Type, tomatoMeter, tomatoImage, tomatoRating, tomatoReviews, tomatoFresh, tomatoRotten, tomatoConsensus, tomatoUserMeter, tomatoUserRating, tomatoUserReviews, tomatoURL, DVD, BoxOffice, Production, Website, Response
 Schema: _c0, Title, Year, Rated, Released, Runtime, Genre, Director, Writer, Actors, Plot, Language, Country, Awards, Poster, Ratings.Source, Ratings.Value, Metascore, imdbRating, imdbVotes, imdbID, Type, tomatoMeter, tomatoImage, tomatoRating, tomatoReviews, tomatoFresh, tomatoRotten, tomatoConsensus, tomatoUserMeter, tomatoUserRating, tomatoUserReviews, tomatoURL, DVD, BoxOffice, Production, Website, Response
Expected: _c0 but found: 
CSV file: file:///Users/dcku/Desktop/2420%20DSF/Lab/Lab11/IMD

# Let's use Spark's SQL to perform operations on PySpark's dataframe. 

# Queries: select and Like functions


In [8]:
imdb_df.select("Title","Year","Rated").show(5)  # view the first five rows based on selected columns

+--------------------+----+--------+
|               Title|Year|   Rated|
+--------------------+----+--------+
|The Shawshank Red...|1994|       R|
|       The Godfather|1972|       R|
|The Godfather: Pa...|1974|       R|
|     The Dark Knight|2008|   PG-13|
|        12 Angry Men|1957|APPROVED|
+--------------------+----+--------+
only showing top 5 rows



In the brackets of the “Like” function, the % character is used to filter out all titles having the “ THE ” word. If the condition we are looking for is the exact match, then no % character shall be used.

In [9]:
imdb_df.select( "Title",
imdb_df.Title.like("%Godfather%")).show(15)

+--------------------+----------------------+
|               Title|Title LIKE %Godfather%|
+--------------------+----------------------+
|The Shawshank Red...|                 false|
|       The Godfather|                  true|
|The Godfather: Pa...|                  true|
|     The Dark Knight|                 false|
|        12 Angry Men|                 false|
|    Schindler's List|                 false|
|The Lord of the R...|                 false|
|        Pulp Fiction|                 false|
|          Fight Club|                 false|
|The Lord of the R...|                 false|
|        Forrest Gump|                 false|
|Star Wars: Episod...|                 false|
|           Inception|                 false|
|The Lord of the R...|                 false|
|One Flew Over the...|                 false|
+--------------------+----------------------+
only showing top 15 rows



# Inspection: distinct() and show()

In [10]:
imdb_df.select("Rated").distinct().show()
#The distinct() will come in handy when you want to determine the unique values in the categorical columns in the dataframe.

+---------+
|    Rated|
+---------+
|   PASSED|
| APPROVED|
|       PG|
|    TV-MA|
|        R|
|NOT RATED|
|        G|
|       GP|
|  UNRATED|
|    PG-13|
+---------+



# Multiple View: groupBy function 

Can we know how many rows of data associated with each rating category ? We can use the groupBy function to group the dataframe column values and then apply an aggregate function on them to derive some useful insight.



In [11]:
from pyspark.sql import functions as F #you can create alias
imdb_df.groupBy("Rated").agg(F.count("Rated").alias("Total-Row")).sort(F.desc("Total-Row")).show()


+---------+---------+
|    Rated|Total-Row|
+---------+---------+
|        R|      104|
|       PG|       39|
|    PG-13|       34|
| APPROVED|       30|
|        G|       16|
|NOT RATED|       15|
|   PASSED|        5|
|  UNRATED|        5|
|    TV-MA|        1|
|       GP|        1|
+---------+---------+



# Data Cleaning

Can data cleaning operations be applied to Pyspark dataframe ? Yes, of course. Let's look at some examples of data cleaning operations. We will use the Automobile data from Lab04.

In [72]:
# pip install numpy

In [73]:
# pip install pandas

In [74]:
# pip install --upgrade pip

In [12]:
import numpy as np
import pandas as pd
cols = ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']
cars = pd.read_csv('imports-85.data.txt', names=cols)

Once we have read the Automobile data, we convert them to Pyspark dataframe.

In [13]:
cars_df = spark.createDataFrame(cars.astype(str))
cars_df.show()

+---------+-----------------+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+
|symboling|normalized-losses|       make|fuel-type|aspiration|num-of-doors| body-style|drive-wheels|engine-location|wheel-base|length|width|height|curb-weight|engine-type|num-of-cylinders|engine-size|fuel-system|bore|stroke|compression-ratio|horsepower|peak-rpm|city-mpg|highway-mpg|price|
+---------+-----------------+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+
|        3|                ?|alfa-romero|      gas|       std|         two|convertible|         rwd|          front|      88.6| 16

From the first glance of the above code, there are many "?""where the data is missing. Let's count the number of "?" in each column of the Pyspark dataframe.

In [14]:
from pyspark.sql import functions as F
cars_df.agg(*[
    F.count(F.when(F.col(str(c)) == "?",1)).alias(c) for c in cars_df.columns]).show()

+---------+-----------------+----+---------+----------+------------+----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+
|symboling|normalized-losses|make|fuel-type|aspiration|num-of-doors|body-style|drive-wheels|engine-location|wheel-base|length|width|height|curb-weight|engine-type|num-of-cylinders|engine-size|fuel-system|bore|stroke|compression-ratio|horsepower|peak-rpm|city-mpg|highway-mpg|price|
+---------+-----------------+----+---------+----------+------------+----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+
|        0|               41|   0|        0|         0|           2|         0|           0|              0|         0|     0|    0|     0|          0|   

The column *normalized-losses* has the most "?". Let's replace the missing values in this column with the most frequently occurred value. The code below helps to identify the frequency of each distinct value in the column *normalized-losses*.

In [15]:
from pyspark.sql import functions as F
cars_df.groupBy("normalized-losses").agg(F.count("normalized-losses").alias("Total")).sort(F.desc("Total")).show()

+-----------------+-----+
|normalized-losses|Total|
+-----------------+-----+
|                ?|   41|
|              161|   11|
|               91|    8|
|              150|    7|
|              104|    6|
|              128|    6|
|              134|    6|
|               85|    5|
|               65|    5|
|              168|    5|
|              103|    5|
|               74|    5|
|              102|    5|
|               95|    5|
|               94|    5|
|              118|    4|
|              148|    4|
|              106|    4|
|               93|    4|
|              122|    4|
+-----------------+-----+
only showing top 20 rows



From the above table, the value 161 is the most frequently occured values in the column normalized-losses. Let's replace the "?" with the value 161. The value 161 must be in the form of a string because the entire PySpark dataframe is of type String. Refer to cell[79].

In [16]:
cars_df = cars_df.replace('?', '161', subset= "normalized-losses")


Checkout whether the replacement is done. Looks like replacement is successful.

In [17]:
cars_df.show()

+---------+-----------------+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+
|symboling|normalized-losses|       make|fuel-type|aspiration|num-of-doors| body-style|drive-wheels|engine-location|wheel-base|length|width|height|curb-weight|engine-type|num-of-cylinders|engine-size|fuel-system|bore|stroke|compression-ratio|horsepower|peak-rpm|city-mpg|highway-mpg|price|
+---------+-----------------+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+
|        3|              161|alfa-romero|      gas|       std|         two|convertible|         rwd|          front|      88.6| 16

There are other columns having "?" as well. However it is not as many as the column *normalized-losses*. Let's just replace it using a default value which is 0. 

In [18]:
cars_df = cars_df.replace('?', '0')
cars_df.show()

+---------+-----------------+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+
|symboling|normalized-losses|       make|fuel-type|aspiration|num-of-doors| body-style|drive-wheels|engine-location|wheel-base|length|width|height|curb-weight|engine-type|num-of-cylinders|engine-size|fuel-system|bore|stroke|compression-ratio|horsepower|peak-rpm|city-mpg|highway-mpg|price|
+---------+-----------------+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+
|        3|              161|alfa-romero|      gas|       std|         two|convertible|         rwd|          front|      88.6| 16

Let's do a check again. Is there any columns having "?" as value ? No, there are no columns having "?" as value. 

In [19]:
cars_df.agg(*[
    F.count(F.when(F.col(str(c)) == "?", 1)).alias(c) for c in cars_df.columns]).show()

+---------+-----------------+----+---------+----------+------------+----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+
|symboling|normalized-losses|make|fuel-type|aspiration|num-of-doors|body-style|drive-wheels|engine-location|wheel-base|length|width|height|curb-weight|engine-type|num-of-cylinders|engine-size|fuel-system|bore|stroke|compression-ratio|horsepower|peak-rpm|city-mpg|highway-mpg|price|
+---------+-----------------+----+---------+----------+------------+----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+
|        0|                0|   0|        0|         0|           0|         0|           0|              0|         0|     0|    0|     0|          0|   

We can convert the column type too. Suppose that we would like to convert the column *symboling* from String to Integer.

In [20]:
from pyspark.sql.types import IntegerType
cars_df = cars_df.withColumn("symboling", F.col("symboling").astype(IntegerType()))

Let's double-check whether the conversion is done. Yes, the conversion is successful. 

In [21]:
cars_df.printSchema()

root
 |-- symboling: integer (nullable = true)
 |-- normalized-losses: string (nullable = true)
 |-- make: string (nullable = true)
 |-- fuel-type: string (nullable = true)
 |-- aspiration: string (nullable = true)
 |-- num-of-doors: string (nullable = true)
 |-- body-style: string (nullable = true)
 |-- drive-wheels: string (nullable = true)
 |-- engine-location: string (nullable = true)
 |-- wheel-base: string (nullable = true)
 |-- length: string (nullable = true)
 |-- width: string (nullable = true)
 |-- height: string (nullable = true)
 |-- curb-weight: string (nullable = true)
 |-- engine-type: string (nullable = true)
 |-- num-of-cylinders: string (nullable = true)
 |-- engine-size: string (nullable = true)
 |-- fuel-system: string (nullable = true)
 |-- bore: string (nullable = true)
 |-- stroke: string (nullable = true)
 |-- compression-ratio: string (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- peak-rpm: string (nullable = true)
 |-- city-mpg: string (nulla

We can create a new column. Let's name the column as *double-symboling*. This new column will multiply the value from the column *symboling* with 2. Use the show() command to check whether the new column and the values have been created / computed. 

In [22]:
cars_df.withColumn("double-symboling", cars_df.symboling*2).show()

+---------+-----------------+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+----------------+
|symboling|normalized-losses|       make|fuel-type|aspiration|num-of-doors| body-style|drive-wheels|engine-location|wheel-base|length|width|height|curb-weight|engine-type|num-of-cylinders|engine-size|fuel-system|bore|stroke|compression-ratio|horsepower|peak-rpm|city-mpg|highway-mpg|price|double-symboling|
+---------+-----------------+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+----+------+-----------------+----------+--------+--------+-----------+-----+----------------+
|        3|              161|alfa-romero|      gas|       std|         two|conv

# Output File

Convert it to pandas dataframe. Once the file is in pandas dataframe, you can use the Python dataframe manipulation operations on it. 

In [24]:
# cars_df.toPandas().to_csv('mycsv.csv', index=False)
cars_df.repartition(1).write.csv('mycsv.csv')

In [25]:
spark.stop()